In [99]:
import requests
import zipfile
from io import BytesIO, TextIOWrapper

import pandas as pd

In [100]:
base_url = "https://data.arr-software.org/"
query_url = base_url+"?lon_coord={0}&lat_coord={1}&type=json&All=1"

lon, lat = (152.648, -29.573)

In [101]:
r = requests.get(query_url.format(lon, lat))
response_json = r.json()

In [102]:
point_tp_url = base_url+response_json['layers']['PointTP']['url']
point_tp_url

'https://data.arr-software.org/static/temporal_patterns/TP/ECsouth.zip'

In [103]:
d = requests.get(point_tp_url, stream=True)
d

<Response [200]>

In [104]:
zipdata = BytesIO()
zipdata.write(d.content)

56304

In [105]:
myzipfile = zipfile.ZipFile(zipdata)
increments_file = [f for f in myzipfile.namelist() if f.endswith("_Increments.csv")][0]

In [106]:
df = pd.read_csv(TextIOWrapper(myzipfile.open(target_file)))
df.columns = [c.strip() for c in df.columns.values]
df["Increments"] = pd.Series(df.iloc[:,5:].values.tolist())
df.Increments = df.Increments.apply(lambda row: [r for r in row if str(r) != 'nan'])
df = df.drop([c for c in df.columns.values if "Unnamed" in c], axis=1)
df.head()

,EventID,Duration,TimeStep,Region,AEP,Increments
0,4380,10,5,East Coast (South),frequent,"[58.06, 41.94]"
1,4382,10,5,East Coast (South),frequent,"[52.13, 47.87]"
2,4384,10,5,East Coast (South),frequent,"[60.71, 39.29]"
3,4385,10,5,East Coast (South),frequent,"[51.51, 48.49]"
4,4386,10,5,East Coast (South),frequent,"[54.55, 45.45]"
